In [1]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
import random
import csv

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = "cuda:0"
model = model.to(device)

#story_one = ["Karen was assigned a roommate her first year of college.", "Her roommate asked her to go to a nearby city for a concert.", "Karen agreed happily.", "The show was absolutely exhilarating.", "Karen became good friends with her roommate."]

/notebooks/pipenv/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 7.81MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 17.2kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 276kB/s]
Downloading: 100%|██████████| 420M/420M [00:18<00:00, 23.5MB/s] 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initiali

In [ ]:
stories = []
with open("100KStories.csv") as file:
    tsv_file = csv.reader(file, delimiter=",")
    for line in tsv_file:
        #print(line)
        line.pop(0)
        #print(line)
        line.pop(0)
        stories.append(line)

In [19]:
shuf_stories = []
for orig_story in stories:
    #print ("BEFORE: ", orig_story)
    sent1_replaced = random.randint(0, 4)
    sent2_replaced = random.randint(0, 4)
    sent3_replaced = random.randint(0, 4)
    story1_replace = random.randint(0, len(stories) -1)
    story2_replace = random.randint(0, len(stories) -1)
    story3_replace = random.randint(0, len(stories) -1)
    orig_story[sent1_replaced] = stories[story1_replace][0]
    orig_story[sent2_replaced] = stories[story2_replace][1]
    orig_story[sent3_replaced] = stories[story3_replace][2]
    #print ("AFTER: ", orig_story)
    shuf_stories.append(orig_story)

In [3]:
def compat_three(s1, s2, s3):
    sent_a = s1+" "+s2
    sent_b = s2+" "+s3
    encoded = tokenizer.encode_plus(sent_a, s3, return_tensors='pt').to(device)
    seq_relationship_logits = model(**encoded)[0]
    probs = softmax(seq_relationship_logits, dim=1)
    compt1 = probs[0][0].tolist()
    encoded = tokenizer.encode_plus(s1, sent_b, return_tensors='pt').to(device)
    seq_relationship_logits = model(**encoded)[0]
    probs = softmax(seq_relationship_logits, dim=1)
    compt2 = probs[0][0].tolist()
    return compt1*compt2


def story_compatability(story):
    result = 1.
    for i in range(len(story)-1):
        sent_a = story[i]
        sent_b = story[i+1]
        encoded = tokenizer.encode_plus(sent_a, sent_b, return_tensors='pt').to(device)
        seq_relationship_logits = model(**encoded)[0]
        probs = softmax(seq_relationship_logits, dim=1)
        compt = probs[0][0].tolist()
        result *= compt
    return result


def story_compatability_3(story):
    result = 1.
    for i in range(len(story)-2):
        sent_a = story[i]
        sent_b = story[i+1]
        sent_c = story[i+2]
        compt = compat_three(sent_a,sent_b,sent_c)
        result *= compt
    return result

In [ ]:
from tqdm import tqdm
result1 = 0
result2 = 0
for story_one in tqdm(shuf_stories[:(len(stories) -1)]):
    #print("Story ", story_one)
    #input()
    #shuf_story = story_one.copy()
    #random.shuffle(shuf_story)
    prob1 = story_compatability(story_one)
    #if prob1 < 0.1:
        #print("PROB1 ", story_one, " ", prob1)
    #print(prob)
    #shuf_prob = story_compatability(shuf_story)
    #print(shuf_prob)
    prob2 = story_compatability_3(story_one)
    #if prob2 < 0.1:
        #print("PROB2 ", story_one, " ", prob2)
    result1 += prob1
    result2 += prob2
    #print(prob1)
    #print(prob2)
    #shuf_prob = story_compatability_3(shuf_story)
    #print(shuf_prob)
print (result1/(len(stories) -1))
print (result2/(len(stories) -1))

  0%|          | 284/98166 [00:45<4:10:56,  6.50it/s]

In [5]:
#print(shuf_story)

In [ ]:
from tqdm import tqdm
result1 = 0
result2 = 0
for story_one in tqdm(stories[:(len(stories) -1)]):
    #print("Story ", story_one)
    #input()
    #shuf_story = story_one.copy()
    #random.shuffle(shuf_story)
    prob1 = story_compatability(story_one)
    #if prob1 < 0.1:
        #print("PROB1 ", story_one, " ", prob1)
    #print(prob)
    #shuf_prob = story_compatability(shuf_story)
    #print(shuf_prob)
    prob2 = story_compatability_3(story_one)
    #if prob2 < 0.1:
        #print("PROB2 ", story_one, " ", prob2)
    result1 += prob1
    result2 += prob2
    #print(prob1)
    #print(prob2)
    #shuf_prob = story_compatability_3(shuf_story)
    #print(shuf_prob)
print (result1/(len(stories) -1))
print (result2/(len(stories) -1))